In [9]:
import numpy as np
import torch
import sys
sys.path.append('../')
from voting_games.werewolf_env_v0 import plurality_env, pare, Phase, Roles
import random
import copy
from tqdm import tqdm
from collections import Counter
from torchview import draw_graph
import graphviz
graphviz.set_jupyter_format('png')
import matplotlib.pyplot as plt
from notebooks.learning_agents.models import ActorCriticAgent
from notebooks.learning_agents.utils import play_recurrent_game, convert_obs
from notebooks.learning_agents.static_agents import random_approval_wolf, random_plurality_wolf
import notebooks.learning_agents.stats as indicators 

'png'

# Training Agents 

## Hyperparameters

There are just under 20 hyperparameters, and can be split between gameplay parameters, neural network model parameters and PPO training parameters. Each value in this configuration object below can be changed, and is passed throughout training functions.

INitializations and hyper-parameter choices were greatly influenced by the exhaustive work in {cite}`Andrychowicz2020-fs` and {cite}`pleines2022memory` for the LSTM portion.


```python
config_training = {
    "model": {
        "recurrent_layers": 1,
        "recurrent_hidden_size": 256, 
        "joint_mlp_size": 128,
        "split_mlp_size": 128,
        "num_votes": 10,
        "approval_states": 3,
    },
    "training" : {
        "batch_size": 256, 
        "epochs": 5, 
        "updates": 501, 
        "buffer_games_per_update": 500, 
        "clip_range": 0.1, 
        "value_loss_coefficient": 0.1, 
        "max_grad_norm": 0.5, 
        "beta": 0.01, 
        "learning_rate": 0.0001, 
        "adam_eps": 1e-5, 
        "gamma": 0.99, 
        "gae_lambda": 0.95, 
    }
}


config_game = {
    "rewards": {
        "day": -1,
        "player_death": -1,
        "player_win": 10,
        "player_loss": -5,
        "self_vote": -1,
        "dead_villager": -1,
        "dead_vote": -1,
        "dead_wolf": 5,
        "no_viable_vote": -1,
        "no_sleep": -1,
    },
    "gameplay": {
        "accusation_phases": 3,
        "num_agents": 10,
        "num_werewolves": 2,
    }
}

config = {
    "config_game": config_game,
    "config_training": config_training,
}
```

## Training w/ PPO